# Missing Data Mechanism
To handle missing data, you need to know the mechanism behind them. You can classify mechanisms of missing data into three categories: Missing Completely at Random (MCAR), Missing at Random (MAR), and Missing Not at Random (MNAR).
Let's dig into their definitions.

### - Missing Completely at Random (MCAR)
There is no systematic difference between observed and missing data. For example, missing data exsits in blood pressure variable because of a machine to measure it breaks down for a while. 

### - Missing at Random (MAR)
Any systematic diference bewteen observed and missing data can be explained by observed data. For example, missing bloog presure data may be distributed with lower mean gaussian distirbution. This happens because young people may be likely not to take tests regulary.

### - Missing Not at Random (MNAR)
Even after considering observed data, the systematic difference is not fully expalined. For example, people wiht high income may hasitate to fill their income at questionnair. 

Roughly speaking, MCAR is dealt with simple statistics like mean or median while predictive models regressed on observed data assumes MAR. MNAR is generally more difficult to deal with. You need to analyze more precisely for each case. If you use a modle whose assumptions are not satisfied, you would introduce bias into data. 

You can see the definition at the paper [[1]](https://www.bmj.com/content/338/bmj.b2393). There is also an interesting paper in the dialog form claryfing the difference between MCAR and MAR [[2]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4121561/).

Here is tips as to how to diagnose missing mechanisms at [this blog post](https://www.theanalysisfactor.com/missing-data-mechanism/).



# Deletion
### - Listwise
Assumption: MCAR
Removes an instance if it contains missing values. 

### - Pairwise
Assumption: MCAR
Keep only cases containing interest varaiables and remove the others. Compared to listwise deletion, pairwise deletion may contain instances having missing data on some variables.

### - Dropping Variables
Assumption: Dropped varialbes do not have any predictive power.
Drop varaibles with missing values.


# Imputation
### - Mean, Median and Mode
Assumption: MCAR
Replace missing values with mean, median or mode. They are one of the most simple imputations and implemented in scikit-learn.

### - Multiple Imputation
Assumption: MAR
Imputation is executed in the following:

1. Impute missing values with basic methods like mean or median
2. Set back imputed values to massing values for each feature to impute
3. The feature is regressed on the other variables, which may or may not include all of other variables
4. Replace missing values with predicted values
5. Repeate 2-4 for each feature
6. Repeat process from 2-5 for the number of cycles. Then, store the last values as an imputed dataset
7. Repeate processe 2-6 to get multiple datasets, which will be used to esitimate uncertainty of the imputation

This imputation is know to work well for small number of features and instance datasets.
Here is the [reference](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/pdf/nihms267760.pdf)


### - K Nearest Neighbor
Assumption: MAR
The efficiency is $O(N^2)$.
You can use [LSH](https://www.slaney.org/malcolm/yahoo/Slaney2008-LSHTutorial.pdf) instead for large scale datasets.
Requires standardization

### - Linear Regression(Logistic Regression)
Assumption: MAR
Regress missing features on the other variables.


In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("train.csv")

In [3]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [4]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000
